In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image

# Load CSV file
data_path = "C:\\Users\\MSI1\\Documents\\assesment\\archive"
csv_file = os.path.join(data_path, "styles.csv")
df = pd.read_csv(csv_file, on_bad_lines='skip')

# Filter necessary columns
columns = ["id", "gender", "articleType", "baseColour", "season"]
df = df[columns]
df.dropna(inplace=True)


In [ ]:


# Define label encoders
from sklearn.preprocessing import LabelEncoder
encoders = {}
for col in columns[1:]:
    encoders[col] = LabelEncoder()
    df[col] = encoders[col].fit_transform(df[col])

# Define Custom Dataset
class FashionDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, str(row["id"]) + ".jpg")

       
        if not os.path.exists(img_path):
            return None  

        image = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row[1:].values, dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, labels


# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create dataset and DataLoader
img_dir = os.path.join(data_path, "images")
dataset = FashionDataset(df, img_dir, transform=transform)

def collate_fn(batch):
    batch = [b for b in batch if b is not None]  # Remove None entries
    if len(batch) == 0:
        return None, None  # Handle case where all images in a batch are missing
    return torch.stack([b[0] for b in batch]), torch.stack([b[1] for b in batch])

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)


# Define Model
class MultiLabelCNN(nn.Module):
    def __init__(self, num_classes_list):
        super(MultiLabelCNN, self).__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)
        self.base_model.classifier = nn.Identity()  # Remove last layer
        
        self.fc_gender = nn.Linear(1280, num_classes_list[0])
        self.fc_articleType = nn.Linear(1280, num_classes_list[1])
        self.fc_baseColour = nn.Linear(1280, num_classes_list[2])
        self.fc_season = nn.Linear(1280, num_classes_list[3])
    
    def forward(self, x):
        x = self.base_model(x)
        return {
            "gender": self.fc_gender(x),
            "articleType": self.fc_articleType(x),
            "baseColour": self.fc_baseColour(x),
            "season": self.fc_season(x)
        }

# Initialize model
num_classes_list = [len(encoders[col].classes_) for col in columns[1:]]
model = MultiLabelCNN(num_classes_list)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
def train_model(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            loss = sum(criterion(outputs[key], labels[:, i]) for i, key in enumerate(outputs))
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")


# Start training
train_model(model, dataloader, criterion, optimizer, epochs=3)

c:\Users\MSI1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MSI1\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/3], Loss: 3.2783


KeyboardInterrupt: 

In [4]:
torch.save(model, "fashion_model.pth")


In [16]:
import torch

# Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Re-initialize the model architecture
num_classes_list = [len(encoders[col].classes_) for col in ["gender", "articleType", "baseColour", "season"]]

# Define the model again
model = MultiLabelCNN(num_classes_list)  # Ensure MultiLabelCNN is defined
model.load_state_dict(torch.load("fashion_model.pth", map_location=device))  # Load weights
model.to(device)  # Move model to the correct device
model.eval()  # Set to evaluation mode

print("Model loaded successfully on", device)


Model loaded successfully on cpu


In [22]:

model = MultiLabelCNN(num_classes_list)  # Initialize the model
model.load_state_dict(torch.load("fashion_model.pth", map_location=torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")))

model.to(device)
model.eval()  # Set to evaluation mode

# Define transformations (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Specify the path of the image you want to test
test_img_path = "10005.jpg"  # Change this to the actual image path

# Load and preprocess the image
image = Image.open(test_img_path).convert("RGB")
# Add batch dimension and move to device
image = transform(image).unsqueeze(0).to(device)
# Perform inference
with torch.no_grad():
    output = model(image)

# Convert output to class labels
predictions = {key: torch.argmax(output[key], dim=1).item() for key in output}

# Decode predictions using label encoders with correct column names
decoded_predictions = {
    "gender": encoders["gender"].inverse_transform([predictions["gender"]])[0],
    # Updated key
    "articleType": encoders["articleType"].inverse_transform([predictions["articleType"]])[0],
    # Updated key
    "baseColour": encoders["baseColour"].inverse_transform([predictions["baseColour"]])[0],
    "season": encoders["season"].inverse_transform([predictions["season"]])[0]
}

print("Predicted Labels:", decoded_predictions)

Predicted Labels: {'gender': 2, 'articleType': 134, 'baseColour': 1, 'season': 0}
